<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Explanations/blob/main/notebooks/train/Sequencer2D_Q600G4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%time
! unzip -q drive/MyDrive/UNC/S2022P/GBIF_Q_20_fourth.zip -d /content/temp
! mv /content/temp/content/dataset/train /content/train
! rm -rf /content/temp

CPU times: user 45.8 ms, sys: 19.9 ms, total: 65.7 ms
Wall time: 5.76 s


In [2]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10841, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 10841 (delta 207), reused 264 (delta 160), pack-reused 10473
Receiving objects: 100% (10841/10841), 20.49 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (7922/7922), done.


In [3]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [6]:
from glob import glob

print(len(list(glob("train/*"))))

60


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py train --model sequencer2d_l --opt adabelief --lr 0.0001 --epochs 80 --aa v0r-mstd0.5 --aug-splits 2 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 4 --input-size 3 600 600 --num-classes=60 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/S2022P/output/ 

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth)
Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth
Model sequencer2d_l created, param count:53936316
Data processing configuration for current model + dataset:
	input_size: (3, 600, 600)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225